* Think of a word or words that define each point  
    * Proximity to each of those words creates a new centroid.
    * proximity is determined using pre-trained word2vec
    * basically, check how close a new word is to each of the centroids (which are defined by words)
    * how close it is to each centroid contributes contributes to it's distance in that direction

In [1]:
import pandas as pd

import pickle

from scipy.spatial import minkowski_distance

Find songs closest to each centroid and define those moods

In [2]:
pickle_in = open('../Data/cluster_centers_tfidf_12', 'rb')
centroids = pickle.load(pickle_in)
pickle_in.close()

In [3]:
df = pd.read_pickle('../Data/labeled_points_tf-idf')

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [5]:
features = df.select_dtypes(include=numerics)
no_features = df.select_dtypes(exclude=numerics)

In [6]:
centroids[0]

array([ 0.16984652, -0.14376436, -0.09898879, -0.10107818, -0.18814935,
       -0.07898272, -0.14817118,  1.32911611, -0.17950391, -1.32395191,
       -0.12853857, -0.30192205, -1.11115323, -0.21473352, -0.7111159 ])

In [32]:
def match_sentiment(sentiment_df, feats_to_match):
    distances_df = sentiment_df.apply(lambda x:
                                      minkowski_distance(x.values,
                                                         feats_to_match, p=1),
                                      axis=1)
    distances_df = pd.DataFrame(distances_df)
    distances_df.columns = ['distance']
    n_smallest = distances_df.nsmallest(15, 'distance').index
    return n_smallest

In [33]:
n_smallest = match_sentiment(features.drop('labels', axis=1), centroids[0])

In [34]:
df.head()

,spotify_album_uri,spotify_artist_id,artist_name,spotify_artist_uri,explicit,spotify_song_id,song_title,song_spotify_page,spotify_song_uri,unaltered_song_title,...,6,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,labels
0,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,14H7ag1wpQOsPPQJOD6Dqr,zero,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,False,2dqpsVfEE3SWltmZ7UWb0m,i have seen,https://open.spotify.com/track/2dqpsVfEE3SWltm...,spotify:track:2dqpsVfEE3SWltmZ7UWb0m,I Have Seen,...,4.174613,-0.890761,-0.154422,0.228150,1.654836,-0.464103,0.117052,-0.391855,1.152411,11
1,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,14H7ag1wpQOsPPQJOD6Dqr,zero,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,False,3C6fiZI9I8xEMbUSfXfQCD,simple things,https://open.spotify.com/track/3C6fiZI9I8xEMbU...,spotify:track:3C6fiZI9I8xEMbUSfXfQCD,Simple Things,...,-0.550689,0.690413,-1.179836,-0.861343,0.354709,-0.393336,-0.037373,-0.434470,-0.875575,0
2,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,14H7ag1wpQOsPPQJOD6Dqr,zero,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,False,0gBTPgvzYPMPbYlTuPjx9n,in the waiting line,https://open.spotify.com/track/0gBTPgvzYPMPbYl...,spotify:track:0gBTPgvzYPMPbYlTuPjx9n,In The Waiting Line,...,-0.550689,1.493938,0.344968,-1.029955,0.304998,-0.573470,-0.862051,-0.479022,-0.245918,0
3,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,14H7ag1wpQOsPPQJOD6Dqr,zero,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,False,0QvF5Q3dg3TpRsU6Wqz9EV,this world,https://open.spotify.com/track/0QvF5Q3dg3TpRsU...,spotify:track:0QvF5Q3dg3TpRsU6Wqz9EV,This World,...,-0.550689,-0.277539,-1.293031,-0.601940,1.563062,-0.560603,0.030541,-0.296941,-1.663464,5
4,spotify:album:4vQcWE7Ce6ORsatrqXkBtt,14H7ag1wpQOsPPQJOD6Dqr,zero,spotify:artist:14H7ag1wpQOsPPQJOD6Dqr,False,3G5qMSP9AywIErX4HSbP5a,spinning,https://open.spotify.com/track/3G5qMSP9AywIErX...,spotify:track:3G5qMSP9AywIErX4HSbP5a,Spinning,...,-0.295682,-0.578473,0.617967,-0.377124,-0.497945,-0.464103,0.054797,-0.479022,-0.617988,8


In [35]:
for i in n_smallest:
    print(df.iloc[i].uri, ',',
          df.iloc[i].unaltered_artist_name, ',',
          df.iloc[i].unaltered_song_title,
          '\n')

spotify:track:26COgYlbcouIcJo3nprpox , Regina Spektor , Laughing With 

spotify:track:5PEpq0XThGwhw8nOTvyUhm , Electrelane , The Valleys 

spotify:track:4jEdyEbhvHZZVLoNHAvIGT , Sun Kil Moon , Moorestown 

spotify:track:4BuSZKTSGqUc5LzdLjBgkA , New Buffalo , It's True 

spotify:track:3UT4KKWTWP1avHXcup7Wag , Emilíana Torrini , Easy - In the Sunshine Acoustic 

spotify:track:13LcTfTWaNHGDgZHbxTaxk , Elliott Smith , Punch and Judy 

spotify:track:4mBMtS5gcTFjH7yEIo9Sfo , Mirah , Advisory Committee 

spotify:track:5vbZ9tprmSSPIPtWJy7LE3 , The Chameleons , All Around 

spotify:track:3pazhNjcz3anIg0VxRDt8f , Great Lake Swimmers , Don't Leave Me Hanging 

spotify:track:17VHkJ5zeQBRHL3i0J0jT8 , Passenger , Fear of Fear 

spotify:track:44lXDWCKq15QMTmbTdQzfx , Cat Power , Lived In Bars 

spotify:track:3mimN4GC3EW0bgbSZn7QKh , The Lucksmiths , Abdication! 

spotify:track:2EdD65IETiGAubrOuFTOTy , Lisa Mitchell , The Land Beyond the Front Door 

spotify:track:5m8gFV8hXan4DmvzUEAczI , Of Monsters 

In [ ]:
for i in n_smallest:
    print(df.iloc[i].uri,'\n')

In [ ]:
centroid[0] - forlorn - misses someone

In [37]:
features.head()

,0,1,2,3,4,5,6,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,labels
0,-0.303712,-0.377076,-0.432827,-0.151821,2.536412,-0.118911,4.174613,-0.890761,-0.154422,0.228150,1.654836,-0.464103,0.117052,-0.391855,1.152411,11
1,0.312279,-0.377076,-0.355183,-0.151821,-0.434343,-0.118911,-0.550689,0.690413,-1.179836,-0.861343,0.354709,-0.393336,-0.037373,-0.434470,-0.875575,0
2,1.004277,-0.370432,-0.432827,-0.132050,-0.155006,-0.102683,-0.550689,1.493938,0.344968,-1.029955,0.304998,-0.573470,-0.862051,-0.479022,-0.245918,0
3,-0.044287,-0.377076,2.264122,-0.151821,-0.671212,-0.118911,-0.550689,-0.277539,-1.293031,-0.601940,1.563062,-0.560603,0.030541,-0.296941,-1.663464,5
4,1.014181,-0.369371,-0.338077,-0.134270,-0.671212,-0.118911,-0.295682,-0.578473,0.617967,-0.377124,-0.497945,-0.464103,0.054797,-0.479022,-0.617988,8
